# TPG366 Pressure Over Time Test

Simple notebook for monitoring pressure over time from a Pfeiffer TPG366 gauge (S/N: 44680058).

## 1. Imports

In [ ]:
import sys
import os
import time
import logging
from datetime import datetime
from pathlib import Path

from bokeh.io import output_notebook, show, push_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, DatetimeTickFormatter

sys.path.append(os.path.join(os.getcwd(), '..', '..', 'src'))

from devices.pfeiffer.tpg366.tpg366 import TPG366

print("Imports done.")

## 2. Setup Logger

In [ ]:
repo_root = Path(os.getcwd()).parent.parent
log_dir = repo_root / "debugging" / "logs"
log_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
log_file = log_dir / f"013_tpg366_pressure_test_{timestamp}.log"

logger = logging.getLogger(f"TPG366_PressureTest_{timestamp}")
logger.setLevel(logging.DEBUG)

file_handler = logging.FileHandler(log_file)
file_handler.setFormatter(logging.Formatter("%(asctime)s - %(levelname)s - %(message)s"))
logger.addHandler(file_handler)

console_handler = logging.StreamHandler()
console_handler.setFormatter(logging.Formatter("%(asctime)s - %(levelname)s - %(message)s"))
logger.addHandler(console_handler)

logger.info("Logger initialized.")
print(f"Log file: {log_file}")

## 3. Initialize and Connect TPG366

In [ ]:
gauge = TPG366(
    device_id="tpg366_44680058",
    port="",  # TODO: set COM port
    device_address=1,
    logger=logger,
)
gauge.connect()
print("TPG366 connected.")

## 4. Verify Connection

In [ ]:
print(f"Serial Number:    {gauge.get_serial_number()}")
print(f"Device Name:      {gauge.get_device_name()}")
print(f"Firmware Version: {gauge.get_firmware_version()}")

## 5. Setup Bokeh Live Plot

In [ ]:
output_notebook()

source = ColumnDataSource(data={"time": [], "pressure": []})

fig = figure(
    title="TPG366 CH1 Pressure Over Time",
    x_axis_label="Time",
    y_axis_label="Pressure (hPa)",
    x_axis_type="datetime",
    width=800,
    height=400,
)
fig.line("time", "pressure", source=source, line_width=2)
fig.xaxis.formatter = DatetimeTickFormatter(hourmin="%H:%M", seconds="%H:%M:%S")

handle = show(fig, notebook_handle=True)

## 6. Live Pressure Monitoring (Interrupt with Stop Button)

In [ ]:
gauge.stop_housekeeping()

## 7. Disconnect

In [ ]:
gauge.disconnect()